In [36]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
import re
from openpyxl.comments import Comment
import time
from utilities import *

start_time = time.time()

In [37]:
COLOR_MODIFIED = PatternFill(start_color="e3ac36", end_color="D3D3D3", fill_type="solid")
font = Font(color='FFFFFF')

In [38]:
old_path =  'Test_Data/BOM/bom_ref.xlsx'
new_path = 'Test_Data/BOM/bom_new.xlsx'
output = 'Test_Data/BOM/bom_output.xlsx'

In [39]:
old_path =  'yazaki_data/BOM/MMSTA LOWDASH DHN BENITA_bom.XLSX'
new_path = 'yazaki_data/BOM/new.xlsx'
output = 'yazaki_data/BOM/output.xlsx'

In [40]:
df_old = pd.read_excel(old_path,engine='openpyxl')
df_new = pd.read_excel(new_path,engine='openpyxl')

In [41]:
# zero_indices = df_old[df_old["level"] == 0].index
# zero_indices
# def get_section(idx,df):
#     previous_zero_idx = zero_indices[zero_indices <= idx].max()
#     return df.loc[previous_zero_idx:idx]

def get_idx_racine(idx_next,df):
    if df.iloc[idx_next].Level == 0 :
        return [idx_next]
    pr_level = df.iloc[idx_next].Level
    pr_level = re.findall(r'\d+', str(pr_level))
    pr_level = pr_level[0]
    # print(pr_level)
    
    idx_levels = [idx_next]
    idx_next = idx_next + 1
    while idx_next < len(df.index):
      nv = df.iloc[idx_next].Level
      nbr = re.findall(r'\d+', str(nv))[0]
      # print(nbr)
      if int(nbr) > int(pr_level) :
        idx_levels.append(idx_next)
      else:
        break
      idx_next = idx_next + 1
    return idx_levels

In [42]:
df_old.head(10)

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status
0,0,MA13,YFG,F000292917,240105971V,6760379.0,2,NaN,1,0,...,A111,NaN,EX,0,0,NaN,L,NaN,0,NaN
1,*1,MA13,YSFG,S008548980,Super Group 020 See drawing,6367154.0,2,NaN,1,0,...,A213,NaN,YX,0,25,NaN,L,PC,1,NaN
2,**2,MA13,YSFG,S008548890,Twisted Wire 26 ZZ,6367066.0,2,NaN,1,0,...,A213,NaN,YX,0,25,NaN,L,PC,1,NaN
3,***3,MA13,YSFG,S008548501,Circuit 1040 COFLRYB 0.35 L/W,6366678.0,92,NaN,92,0,...,A213,NaN,YX,0,25,M,L,PC,1,NaN
4,****4,MA13,YDM,FW406994,LT Single Wire COFLRYB 0.35 L/W,NaN,92,NaN,0,0,...,A305,NaN,ZK,0,17000,NaN,L,MM,1595,NaN
5,****4,MA13,YDM,7195028702,Terminal 025 Male 0.64mm Unsealed,NaN,92,NaN,0,0,...,A305,NaN,ZK,0,60000,L,L,PC,1,NaN
6,***3,MA13,YSFG,S008548500,Circuit 1039 COFLRYB 0.35 BR/W,6366677.0,92,NaN,92,0,...,A213,NaN,YX,0,25,M,L,PC,1,NaN
7,****4,MA13,YDM,FW406984,LT Single Wire COFLRYB 0.35 BR/W,NaN,92,NaN,0,0,...,A305,NaN,ZK,0,17000,NaN,L,MM,1595,NaN
8,****4,MA13,YDM,7195028702,Terminal 025 Male 0.64mm Unsealed,NaN,92,NaN,0,0,...,A305,NaN,ZK,0,60000,L,L,PC,1,NaN
9,**2,MA13,YSFG,S008548889,Twisted Wire 28 ZZ,6367065.0,2,NaN,1,0,...,A213,NaN,YX,0,25,NaN,L,PC,1,NaN


In [43]:
get_idx_racine(2,df_old)

[2, 3, 4, 5, 6, 7, 8]

In [44]:
columns = list(df_old.columns)
columns

['Level',
 'Plant',
 'Material Type',
 'Material',
 'Material Description',
 'Bill of material',
 'Plant-sp.matl status',
 'Plant-sp.matl status.1',
 'BOM status',
 'BOM status.1',
 'Change Number',
 'Changed on',
 'MRP Controller',
 'MRP Controller.1',
 'MRP profile',
 'MRP profile.1',
 'Lot size',
 'Minimum Lot Size',
 'Rounding value',
 'Spare part indicator',
 'Item Category',
 'Component unit',
 'Component quantity',
 'Status']

In [45]:
df_new.shape, df_old.shape

((116825, 24), (116825, 24))

In [46]:
df_old = df_old.fillna('NAN')
df_new = df_new.fillna('NAN')

In [47]:
df_old['Changed on'] = df_old['Changed on'].astype(str)
df_new['Changed on'] = df_new['Changed on'].astype(str)

In [48]:
df_new.ne(df_old)

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116820,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
116821,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
116822,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
116823,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [49]:
diff_values = df_old.where(df_new.ne(df_old), other="SAME")
diff_values

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status
0,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
1,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
2,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
3,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
4,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116820,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
116821,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
116822,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME
116823,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,...,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME,SAME


In [58]:
diff_values.index[diff_values.ne("SAME").any(axis=1)]

Index([9, 17], dtype='int64')

In [52]:
diff_values.ne("SAME").any(axis=1).value_counts()

False    116823
True          2
Name: count, dtype: int64

In [53]:
df_diff = pd.merge(df_old, df_new, how='right', on = list(df_old.columns), indicator=True)

In [54]:
df_diff = df_diff[df_diff['_merge'] == 'right_only'].drop(columns=['_merge'])

In [55]:
idx_diff = df_diff.index
df_diff

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status
1705,**2,MA13,YSFG,S008548889,Twisted Wire 28 ZZ,6367065.0,23,NAN,1,0,...,A213,NAN,YX,0,25,NAN,L,PC,1,NAN
3345,***3,MA13,YSFG,S008548497,Circuit 1126 COFLRYB 0.35 BR/W,6366674.0,92,NAN,92,0,...,A213,NAN,yyy,0,25,M,L,PC,1,NAN


In [59]:
idx_diff = diff_values.index[diff_values.ne("SAME").any(axis=1)]

In [60]:
COLOR_MODIFIED = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
COLOR_MODIFIED_section = PatternFill(start_color="e3ac36", end_color="D3D3D3", fill_type="solid")

font = Font(color='FFFFFF')

wb_new = load_workbook(new_path)
ws_new = wb_new.active

if "Summary" not in wb_new.sheetnames:
    ws_summary = wb_new.create_sheet("Summary")
else:
    ws_summary = wb_new["Summary"]

ws_summary.delete_rows(1, ws_summary.max_row)

for idx,col in enumerate(df_old.columns):
    cell = f'A{idx+2}'
    ws_summary[cell] = col


row_sum, col_sum = 1,2

for row_idx in idx_diff :
    for col_idx in range(1, df_new.shape[1]):
        old_value = df_old.iloc[row_idx,col_idx]
        new_value = df_new.iloc[row_idx,col_idx]
        
        if pd.isna(old_value) and pd.isna(new_value):                
            continue
        elif old_value != new_value :
            print(f'old value {old_value} ,new value {new_value}')
            cell = ws_new.cell(row=row_idx+2, column=col_idx+1)
            cell.fill = COLOR_MODIFIED
            cell.font = font
            cell.comment = Comment(add_comment(old_value, new_value), "AutoComparer")

            
    section = get_idx_racine(row_idx, df_old)
    print(f'Section : {section}')
    for row in section:
        ws_summary.cell(row=row_sum, column=col_sum, value=f"Row {row+2}")
        for col in range(df_new.shape[1]):
            row_sum += 1
            print(f'row {row}, col {col} //// value {df_new.iloc[row,col]}')
            cell_summary = ws_summary.cell(row=row_sum, column=col_sum, value=df_new.iloc[row,col])
            
            if df_old.iloc[row,col] != df_new.iloc[row,col] :
                cell_summary.fill = COLOR_MODIFIED
                cell_summary.font = font
                cell_summary.comment = Comment(add_comment(df_old.iloc[row,col], df_new.iloc[row,col]), "AutoComparer")
                continue

            cell_summary.fill = COLOR_MODIFIED_section
            cell_summary.font = font
            cell_summary.comment = Comment(add_comment(df_old.iloc[row,col], df_new.iloc[row,col]), "AutoComparer")
            cell = ws_new.cell(row=row+2, column=col+1)
            cell.fill = COLOR_MODIFIED_section
            cell.font = font
        col_sum +=1
        row_sum = 1
        
            
wb_new.save(output)
print(f"Comparison complete. Differences highlighted in {output}.")

old value 2 ,new value 23
Section : [9, 10, 11, 12, 13, 14, 15]
row 9, col 0 //// value **2
row 9, col 1 //// value MA13
row 9, col 2 //// value YSFG
row 9, col 3 //// value S008548889
row 9, col 4 //// value Twisted Wire 28   ZZ
row 9, col 5 //// value 6367065.0
row 9, col 6 //// value 23
row 9, col 7 //// value NAN
row 9, col 8 //// value 1
row 9, col 9 //// value 0
row 9, col 10 //// value 500000250185
row 9, col 11 //// value NaT
row 9, col 12 //// value TH2
row 9, col 13 //// value NAN
row 9, col 14 //// value A213
row 9, col 15 //// value NAN
row 9, col 16 //// value YX
row 9, col 17 //// value 0
row 9, col 18 //// value 25
row 9, col 19 //// value NAN
row 9, col 20 //// value L
row 9, col 21 //// value PC
row 9, col 22 //// value 1
row 9, col 23 //// value NAN
row 10, col 0 //// value ***3
row 10, col 1 //// value MA13
row 10, col 2 //// value YSFG
row 10, col 3 //// value S008548499
row 10, col 4 //// value Circuit 0059 COFLRYB 0.35 L/W
row 10, col 5 //// value 6366676.0
row 10

In [19]:
end_time = time.time()

execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time} secondes")

Temps d'exécution : 695.203408241272 secondes


In [21]:
execution_time / 60

11.586723470687867